In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
import math
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage
from contextlib import closing

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

In [ ]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update -qq #--------------------------------------->to gcp
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'adi3158'
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)


In [ ]:
import sys
import builtins
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import os
import re
from operator import itemgetter
from pathlib import Path
import pickle
from contextlib import closing

BLOCK_SIZE = 1999998

class MultiFileWriter:
    """ Sequential binary writer to multiple files of up to BLOCK_SIZE each. """
    def __init__(self, base_dir, name, bucket_name):#7 #('.',fullname,bucket_name)
        self._base_dir = Path(base_dir)
        self._name = name
        self.client = storage.Client() #new ohad todo
        self._bucket_name = self.client.bucket(bucket_name)
        self._file_gen = (open(self._base_dir / f'{name}_{i:03}.bin', 'wb') 
                          for i in itertools.count())
        self._f = next(self._file_gen)
    
    def write(self, b):#*******************************************
      locs = []
      while len(b) > 0:
        pos = self._f.tell()
        remaining = BLOCK_SIZE - pos
        # if the current file is full, close and open a new one.
        if remaining == 0:  
          self._f.close()
        
          file_name = self._f.name # ohad todo change
          blob = self._bucket_name.blob(f"postings_gcp/{file_name}")
          blob.upload_from_filename(file_name)
            
          self._f = next(self._file_gen)
          pos, remaining = 0, BLOCK_SIZE
        self._f.write(b[:remaining])
        locs.append((self._f.name, pos))
        b = b[remaining:]
      return locs

    def close(self):
      self._f.close()

class MultiFileReader:
  """ Sequential binary reader of multiple files of up to BLOCK_SIZE each. """
  def __init__(self):
    self._open_files = {}
    self.client = storage.Client()
    self._bucket_name = self.client.bucket(bucket_name)

  def read(self, locs, n_bytes):
    b = []
    for f_name, offset in locs:  
      if f_name not in self._open_files:
        blob = self._bucket_name.get_blob(f'postings_gcp/{f_name}')
        self._open_files[f_name] = blob.open('rb')
      f = self._open_files[f_name]
      f.seek(offset)
      n_read = builtins.min(n_bytes, BLOCK_SIZE - offset)
      b.append(f.read(n_read))
      n_bytes -= n_read
    return b''.join(b)
  
  def close(self):
    for f in self._open_files.values():
      f.close()

  def __exit__(self, exc_type, exc_value, traceback):
    self.close()
    return False

TUPLE_SIZE = 6       # We're going to pack the doc_id and tf values in this 
                     # many bytes.
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
# DL = {} 
class InvertedIndex:  
  def __init__(self, docs={}):
    """ Initializes the inverted index and add documents to it (if provided).
    Parameters:
    -----------
      docs: dict mapping doc_id to list of tokens
    """
    # DL represent a dict of {doc_id : len of the doc}
    self.DL = {}
    # stores document frequency per term
    self.df = Counter()
    # stores total frequency per term
    self.term_total = Counter()
    # stores posting list per term while building the index (internally), 
    # otherwise too big to store in memory.
    self._posting_list = defaultdict(list)
    # mapping a term to posting file locations, which is a list of 
    # (file_name, offset) pairs. Since posting lists are big we are going to
    # write them to disk and just save their location in this list. We are 
    # using the MultiFileWriter helper class to write fixed-size files and store
    # for each term/posting list its list of locations. The offset represents 
    # the number of bytes from the beginning of the file where the posting list
    # starts. 
    self.posting_locs = defaultdict(list)
    
    for doc_id, tokens in docs.items():
      self.add_doc(doc_id, tokens)

  def add_doc(self, doc_id, tokens):
    """ Adds a document to the index with a given `doc_id` and tokens. It counts
        the tf of tokens, then update the index (in memory, no storage 
        side-effects).
    """
    DL[(doc_id)] = DL.get(doc_id,0) + (len(tokens))
    w2cnt = Counter(tokens)
    self.term_total.update(w2cnt)
    for w, cnt in w2cnt.items():
      self.df[w] = self.df.get(w, 0) + 1
      self._posting_list[w].append((doc_id, cnt))

  def write_index(self, base_dir, name):
    """ Write the in-memory index to disk. Results in the file: 
        (1) `name`.pkl containing the global term stats (e.g. df).
    """
    self._write_globals(base_dir, name)

  def _write_globals(self, base_dir, name):
    with open(Path(base_dir) / f'{name}.pkl', 'wb') as f:
      pickle.dump(self, f)

  def __getstate__(self):
    """ Modify how the object is pickled by removing the internal posting lists
        from the object's state dictionary. 
    """
    state = self.__dict__.copy()
    del state['_posting_list']
    return state

  def posting_lists_iter(self):
    """ A generator that reads one posting list from disk and yields 
        a (word:str, [(doc_id:int, tf:int), ...]) tuple.
    """
    with closing(MultiFileReader()) as reader:
      for w, locs in self.posting_locs.items():
        
        b = reader.read(locs, self.df[w] * TUPLE_SIZE)
        
        posting_list = []
        for i in range(self.df[w]):
          doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
          tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
          posting_list.append((doc_id, tf))
        yield w, posting_list

  def posting_lists_iter_query(self,query_to_search):
    """ A generator that reads one posting list from disk and yields 
        a (word:str, [(doc_id:int, tf:int), ...]) tuple.
    """
    with closing(MultiFileReader()) as reader:
      for w in query_to_search:
        posting_list = []
        if w in self.posting_locs:
          locs = self.posting_locs[w]
          b = reader.read(locs, self.df[w] * TUPLE_SIZE)
          for i in range(self.df[w]):
            doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
            tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
            posting_list.append((doc_id, tf))
            
        yield w, posting_list

  @staticmethod
  def read_index(base_dir, name):
    with open(Path(base_dir) / f'{name}.pkl', 'rb') as f:
      return pickle.load(f)

  @staticmethod
  def delete_index(base_dir, name):
    path_globals = Path(base_dir) / f'{name}.pkl'
    path_globals.unlink()
    for p in Path(base_dir).rglob(f'{name}_*.bin'):
      p.unlink()


  @staticmethod
  def write_a_posting_list(b_w_pl,prefix): # prefix = indexB / indexT ... 
    ''' Takes a (bucket_id, [(w0, posting_list_0), (w1, posting_list_1), ...]) 
    and writes it out to disk as files named {bucket_id}_XXX.bin under the 
    current directory. Returns a posting locations dictionary that maps each 
    word to the list of files and offsets that contain its posting list.
    Parameters:
    -----------
      b_w_pl: tuple
        Containing a bucket id and all (word, posting list) pairs in that bucket
        (bucket_id, [(w0, posting_list_0), (w1, posting_list_1), ...])
    Return:
      posting_locs: dict
        Posting locations for each of the words written out in this bucket.
    '''
    posting_locs = defaultdict(list)
    bucket, list_w_pl = b_w_pl
    print("in write posting list")
    fullname = prefix + str(bucket)
    
    with closing(MultiFileWriter('.',fullname,bucket_name)) as writer:
      for w, pl in list_w_pl:
        print("pl is " ,pl)
        # convert to bytes
        b = b''.join([(doc_id << 16 | (tf & TF_MASK)).to_bytes(TUPLE_SIZE, 'big') for doc_id, tf in pl])
        print('b is ', b)
        # write to file(s)
        locs = writer.write(b)#****************************** add index name todo new ohad
        #save file locations to index
        posting_locs[w].extend(locs)
        
      file_name = writer._f.name
      blob = writer._bucket_name.blob(f"postings_gcp/{file_name}")
      blob.upload_from_filename(file_name)    
    
    return posting_locs

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)


def tokenize(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.
    
    Parameters:
    -----------
    text: string , represting the text to tokenize.    
    
    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens =  [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords]    
    return list_of_tokens
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  lst =  sorted(unsorted_pl, key=lambda y: (y[0]))
  return lst

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  posting_pair = postings.mapValues(len)
  
  return posting_pair
  #[('political', 290), ('philosophy', 135), ('movement', 231), ('sceptical', 12)
def word_count(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens2=[]
  for word in tokens:
    if word not in all_stopwords:
      tokens2.append(word)
   
  dic = {}
  for word in tokens2:
    if word in dic:
      dic[word]= (id,dic[word][1]+1)
    else:
      dic[word]=(id,1)
  lst =[(k,v) for k,v in dic.items()]
  return lst    
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings,prefix=""):
  #input (bucket_id, [(w0, posting_list_0), (w1, posting_list_1), ...])
  #print(postings.collect()[0]) #: ('political', [(1, 336), (1, 594), (1, 676),...
    
  buckts_id = postings.map(lambda y:(token2bucket_id(y[0]),y)) # buckt id, 
  union_buc = buckts_id.groupByKey()
  locations =  union_buc.map(lambda x : InvertedIndex.write_a_posting_list(x,prefix))
  return locations

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

def tfidf_vector(doc, index):    
    lstDoc = doc
    epsilon = .0000001
    total_vocab_size = len(index.term_total)
    Q = np.zeros(len(lstDoc))
    term_vector = lstDoc   
    counter = Counter(lstDoc)
    for token in np.unique(lstDoc):
        if token in index.df.keys(): # avoid terms that do not appear in the index.   
            tf = counter[token]/len(lstDoc) # term frequency divded by the length of the query
            df = index.df[token]            
            idf = math.log((len(index.DL))/(df+epsilon),10) #smoothing
            try:
                ind = term_vector.index(token)
                Q[ind] = tf*idf               
            except:
                pass
    return np.linalg.norm(Q)

In [ ]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)
parquetFile.count()# should be 6M

In [ ]:
# ------------ body index
doc_body_pairs = parquetFile.select("text", "id").rdd
doc_body_pairs_tok = doc_body_pairs.map(lambda x: (tokenize(x[0]),x[1]))
DL_body = doc_body_pairs_tok.map(lambda x:(x[1],len(x[0]))).collectAsMap()



In [ ]:
word_counts_body = doc_body_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings_body = word_counts_body.groupByKey().mapValues(reduce_word_counts) 
postings_filtered_body = postings_body.filter(lambda x: len(x[1])>50) #50
#body - calc df
w2df_body = calculate_df(postings_filtered_body)
w2df_dict_body = w2df_body.collectAsMap()
posting_locs_list_body = partition_postings_and_write(postings_filtered_body,"indexB").collect()

In [ ]:
super_posting_locs_body = defaultdict(list)
for posting_loc in posting_locs_list_body:
  for k, v in posting_loc.items():
    super_posting_locs_body[k].extend(v)
    

In [ ]:
#body index
# Create inverted index instance
invertedB = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
invertedB.posting_locs = super_posting_locs_body#B
# Add the token - df dictionary to the inverted index
invertedB.df = w2df_dict_body
# Add the DL dict
invertedB.DL = DL_body

# calc tf-idf for each term and doc, before query
doc_body_tfidf = doc_body_pairs_tok.map(lambda x: (x[1],tfidf_vector(x[0],invertedB))).collectAsMap()#new new
invertedB.tfidf_dict = doc_body_tfidf

# write the global stats out
invertedB.write_index('.', 'indexB')

# upload to gs
index_src = "indexB.pkl"

index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst



In [ ]:
#--------------- title index
doc_title_pairs = parquetFile.select("title", "id").rdd
doc_title_pairs_tok = doc_title_pairs.map(lambda x: (tokenize(x[0]),x[1]))
DL_title = doc_title_pairs_tok.map(lambda x:(x[1],len(x[0]))).collectAsMap()


In [ ]:
word_counts_title = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts) 
postings_filtered_title = postings_title.filter(lambda x: len(x[1])>0) #50
#title - calc df
w2df_title = calculate_df(postings_filtered_title)
w2df_dict_title = w2df_title.collectAsMap()
posting_locs_list_title = partition_postings_and_write(postings_filtered_title,"indexT").collect()

super_posting_locs_title = defaultdict(list)
for posting_loc in posting_locs_list_title:
  for k, v in posting_loc.items():
    super_posting_locs_title[k].extend(v)
    

In [ ]:
word_counts_title.take(1)

In [ ]:
#title index
# Create inverted index instance
invertedT = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
invertedT.posting_locs = super_posting_locs_title
# Add the token - df dictionary to the inverted index
invertedT.df = w2df_dict_title
# Add the DL dict
invertedT.DL = DL_title



In [ ]:
doc_title_tfidf = doc_title_pairs_tok.map(lambda x: (x[1],tfidf_vector(x[0],invertedT))).collectAsMap()#new new
invertedT.tfidf_dict = doc_title_tfidf


In [ ]:

# write the global stats out
invertedT.write_index('.', 'indexT')



In [ ]:
# upload to gs
index_src = "indexT.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
#------------------ index anchor
#parq_test = parquetFile[parquetFile["id"].isin(lst)]
doc_anchor_pair = parquetFile.select("id","anchor_text").rdd
doc_anchor_pair_flat = doc_anchor_pair.flatMap(lambda x :x[1]).groupByKey().mapValues(list).map(lambda x : (x[0]," ".join([y for y in x[1]])))
doc_anchor_pairs_tok = doc_anchor_pair_flat.map(lambda x: (x[0],tokenize(x[1])))
DL_anchor = doc_anchor_pairs_tok.map(lambda x:(x[0],len(x[1]))).collectAsMap()



In [ ]:
len(DL_anchor)

5867103

In [ ]:

word_counts_anchor = doc_anchor_pair_flat.flatMap(lambda x: word_count(x[1], x[0]))
postings_anchor = word_counts_anchor.groupByKey().mapValues(reduce_word_counts) 
postings_filtered_anchor = postings_anchor.filter(lambda x: len(x[1])>0) #50

# #anchot - calc df
w2df_anchor = calculate_df(postings_filtered_anchor)
w2df_dict_anchor = w2df_anchor.collectAsMap()
posting_locs_list_anchor = partition_postings_and_write(postings_filtered_anchor,"indexA").collect()
print("end")

super_posting_locs_anchor = defaultdict(list)
for posting_loc in posting_locs_list_anchor:
  for k, v in posting_loc.items():
    super_posting_locs_anchor[k].extend(v)
    

end


In [ ]:

#print(general_anchor.take(1))

#anchor index
# Create inverted index instance
invertedA = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
invertedA.posting_locs = super_posting_locs_anchor
# Add the token - df dictionary to the inverted index
invertedA.df = w2df_dict_anchor
# Add the DL dict
invertedA.DL = DL_anchor

#
doc_anchor_tfidf = doc_anchor_pairs_tok.map(lambda x: (x[0],tfidf_vector(x[1],invertedA))).collectAsMap()#new new
invertedA.tfidf_dict = doc_anchor_tfidf


In [ ]:
for blob in client.list_blobs(bucket_name):
  if not blob.name.endswith("indexT.pkl"):
    continue
  with blob.open("rb") as f:
    invertedT = pickle.load(f)
    print("cretae invertedT")    
    
for blob in client.list_blobs(bucket_name):
  if not blob.name.endswith("indexA.pkl"):
    continue
  with blob.open("rb") as f:
    invertedA = pickle.load(f)
    print("cretae invertedA")
    
for blob in client.list_blobs(bucket_name):
  if not blob.name.endswith("indexB.pkl"):
    continue
  with blob.open("rb") as f:
    invertedB = pickle.load(f)
    print("cretae invertedB")

  

In [ ]:
for blob in client.list_blobs(bucket_name):
  if not blob.name.endswith("page_rank_dict.pckl"):
    continue
  with blob.open("rb") as f:
    page_rank_dict = pickle.load(f)
    print("cretae new pagerank")
    
for blob in client.list_blobs(bucket_name):
  if not blob.name.endswith("pageviews-202108-user.pkl"):
    continue
  with blob.open("rb") as f:
    page_view_dict = pickle.load(f)
    print("cretae new pageview")
    
for blob in client.list_blobs(bucket_name):
  if not blob.name.endswith("title_dict.pkl"):
    continue
  with blob.open("rb") as f:
    title_dict = pickle.load(f)
    print("cretae title dict")    